In [ ]:
import sys
import os

os.chdir('../src')
os.getcwd()

In [ ]:
# import library
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import OneHotEncoder
import sys
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import ElementwiseProblem
from pymoo.factory import (get_crossover, get_mutation, get_sampling,
                           get_termination)
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.variable import Real, Integer, Choice, Binary
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.mixed import MixedVariableMating, MixedVariableGA, MixedVariableSampling, MixedVariableDuplicateElimination
from pymoo.optimize import minimize
from trainers_hkg import Trainers
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import warnings
import shap
import pickle
from sklearn.decomposition import PCA
import seaborn as sns
from featurizer import featurization_FW2, normalizedata, score,shap_plot,plot,transformPCA_ONE,convertInputs



In [ ]:
filename = '/Volumes/Chathura_K/GitHub/ML-MD-Separations/data/generated/1-0-hgk-MD-ActCoeff-processed_Activity.xlsx'

In [ ]:
polymers_names = [
    'AR103',
    'AR204',
    'CR61',
    'NAFION',
    'PVAS_BCE',
    'PVAS_RCE',
    'PSbNMP_BCE',
    'PSbNMP_RCE',
    'CEM1',
    'CEM2',
    'CEM3',
    'XLAPn4_9p',
    'XLAPn4_45p'
    
]

polymers_smiles = [
    'CC1CC2CCC3=CC=C(C=C3)C(CC(CC(C)C4=CC=C1C=C4)C5=CC=C2C=C5)CC(CC(CC(C)C6=CC=C(C[N+](C)(C)C)C=C6)C7=CC=C(C[N+](C)(C)C)C=C7)C8=CC=C(C[N+](C)(C)C)C=C8',
    'CC1CC2(C)CC(C)(CC(C)(CC(C)(CC(C)(C)C(=O)OCC[N+](C)(C)C)C(=O)OCC[N+](C)(C)C)C(=O)OCC[N+](C)(C)C)C(=O)OCCOC(=O)C(C)(C)CC(C)(CC(C)(C)C(=O)OCCOC1=O)C(=O)OCCOC2=O',
    'CC1CC2CCC3=CC=C(C=C3)C(CC(CCC4=CC=C1C=C4)C5=CC=C2C=C5)CC(CC(CC(C)C6=CC=C(C=C6)[S](=O)(=O)[O-])C7=CC=C(C=C7)[S](=O)(=O)[O-])C8=CC=C(C=C8)[S](=O)(=O)[O-]',
    '[O-][S](=O)(=O)C(F)(F)C(F)(F)OC(F)(C(F)(F)F)C(F)(F)OC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(OC(F)(F)C(F)(OC(F)(F)C(F)(F)[S](=O)(=O)[O-])C(F)(F)F)C(F)(F)C(F)(F)C(F)(F)C(F)(OC(F)(F)C(F)(OC(F)(F)C(F)(F)[S](=O)(=O)[O-])C(F)(F)F)C(F)(F)F',
    'CC(O)CC(O)CC(O)CC(CC(CC(C)O[S](=O)(=O)[O-])O[S](=O)(=O)[O-])O[S](=O)(=O)[O-]',
    'CC(CC(O)CC(O)CC(CC(O)CC(C)O[S](=O)(=O)[O-])O[S](=O)(=O)[O-])O[S](=O)(=O)[O-]',
    'CC(CC(CC(CC(CC(CC(CC(CC(CCC1=CC=CC=[N+]1C)C2=NC=CC=C2)C3=CC=CC=[N+]3C)C4=NC=CC=C4)C5=CC=CC=[N+]5C)C6=NC=CC=C6)C7=CC=CC=C7)C8=CC=CC=C8)C9=CC=CC=C9',
    'CC(CC(CC(CC(CC(CC(CC(CC(CC(C)C1=NC=CC=C1)C2=[N+](C)C=CC=C2)C3=CC=CC=C3)C4=[N+](C)C=CC=C4)C5=CC=CC=C5)C6=[N+](C)C=CC=C6)C7=CC=CC=C7)C8=CC=CC=N8)C9=NC=CC=C9',
    'CC(CC(CC(CC1(C)CC2(C)CC(C)(C)C(=O)OCCCOCCOC(=O)C(C)(C)CC(C)(CC(C)(C)C(=O)OCCOCCCOC1=O)C(=O)OCCOCCCOC2=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O',
    'CC(CC(CC1(C)CC2(C)CC3(C)CC(C)C(=O)OCCOCCCOC(=O)C(C)(C)CC(C)(CC(C)(CC(C)(C)C(=O)OCCCOCCOC1=O)C(=O)OCCCOCCOC2=O)C(=O)OCCCOCCOC3=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O',
    'CC(CC1(C)CC2(C)CC3(C)CC4(C)CC(C)(C)C(=O)OCCCOCCOC(=O)C(C)CC(C)(CC(C)(CC(C)(CC(C)(C)C(=O)OCCOCCCOC1=O)C(=O)OCCOCCCOC2=O)C(=O)OCCOCCCOC3=O)C(=O)OCCOCCCOC4=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O',
    'CC(C)(C[S]([O-])(=O)=O)NC(=O)CC1C2C3C4CC(=O)OCCOCCOCCOCCOC(=O)CC(C(C(CC(=O)OCCOCCOCCOCCOC1=O)C(=O)OCCOCCOCCOCCOC2=O)C(=O)OCCOCCOCCOCCOC3=O)C(=O)OCCOCCOCCOCCOC4=O',
    'CC(C(C(C(C1C(C)C(=O)OCCOCCOCCOCCOC(=O)C(C)C(C)C(=O)OCCOCCOCCOCCOC1=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O)C(=O)NC(C)(C)C[S]([O-])(=O)=O'
    
]

polymers_dict_ = dict(zip(polymers_names, polymers_smiles))

#polymers_dict_

In [ ]:
# Load data, encode the ions, molecules
(data01_MorganDescr, ohe_ions01_MorganDescr, fingerprints01_MorganDescr) = featurization_FW2(
    filename, polymers_names, polymers_smiles, descriptorType = 'MorganDescr', approach = 'METHODX_A0')

In [ ]:
X = data01_MorganDescr.iloc[:,:-1]
Y = data01_MorganDescr.iloc[:, -1]

## A0 -No transfomation

In [ ]:
# split and transfrom dataset
# if transformation of input is desired, set property = 'X' and transform = True
# if transformation of output is desired, set property = 'Y' and transform = True
# set splitRatio = 0.2 or 0.1, state = 0
(X01_MF_train, X01_MF_test, y01_MF_train, y01_MF_test) = normalizedata(X, Y, 
                                                        splitRatio = 0.2, state = 0,transform = False, property = 'X')

print(f"X01_train, X01_test, y01_train, and y01_test has shapes of {X01_MF_train.shape}, {X01_MF_test.shape}, {y01_MF_train.shape}, and {y01_MF_test.shape}")

## A1 - PCA apply to all the input features

In [ ]:
# PCA transform

pca_X01_MF_train_ALL, pca_X01_MF_test_ALL = convertInputs(X01_MF_train, X01_MF_test,fingerprints01_MorganDescr, type = 'All')
print(f"pca_X01_MF_train_ALL and pca_X01_MF_test_ALL has shapes of {pca_X01_MF_train_ALL.shape}, and {pca_X01_MF_test_ALL.shape}, respectively.")

In [ ]:
transformPCA_ONE(x = X, length = fingerprints01_MorganDescr.shape[1], type = 'ALL')

## A2 - PCA applied only to the polymer fingerprints

In [ ]:
# PCA transform
pca_X01_MF_train_ALL, pca_X01_MF_test_ALL = convertInputs(X01_MF_train, X01_MF_test,fingerprints01_MorganDescr, type = 'All')
pca_X01_MF_train_MDFP, pca_X01_MF_test_MDFP = convertInputs(X01_MF_train, X01_MF_test,fingerprints01_MorganDescr,type = 'MDFP')
print(f"pca_X01_MF_train_MDFP, pca_X01_MF_test_MDFP, y01_train, and y01_test has shapes of {pca_X01_MF_train_MDFP.shape}, and {pca_X01_MF_test_MDFP.shape}, respectively.")

In [ ]:
transformPCA_ONE(x = X, length = fingerprints01_MorganDescr.shape[1], type = 'MDFP')

In [ ]:
# Load train and test data (A0 / A1 /A2)
X_train =X01_MF_train
y_train =y01_MF_train
X_test =X01_MF_test
y_test =y01_MF_test

## ML models
## Hyperpaermater optimization
## SVR

In [ ]:
class svrModel(ElementwiseProblem):
    def __init__(self, X_train, X_test, y_train, y_test, **kwargs):
        vars = {
            "C": Real(bounds = (0.1, 100)),
            "epsilon": Real(bounds = (0.01, 20.0))
        }

        super().__init__(vars=vars, n_obj=3, n_ieq_constr=0, **kwargs)

        self.trainer = Trainers()

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test


    def _evaluate(self, X, out, *arg, **kwargs):

        scores = self.trainer.trainSVR(h_params= X,
                                       X_train=self.X_train,
                                       X_test=self.X_test,
                                       y_train=self.y_train,
                                       y_test=self.y_test,
                                       folds=5)

        out['F'] = [-1 * scores[0], scores[1], -1 * scores[2]]


class Solvers(ElementwiseProblem):
    def __init__(self):
        super(ElementwiseProblem, self).__init__()

    def svrSolver(self, X_train, X_test, y_train, y_test):

        problem = svrModel(X_train, X_test, y_train, y_test)

        algorithm = NSGA2(  
                        pop_size=2,
                        sampling=MixedVariableSampling(),
                        mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                        eliminate_duplicates=MixedVariableDuplicateElimination(),
                        n_offsprings=4
                        )

        res = minimize(problem,
                       algorithm,
                       termination=("n_gen", 500),
                       seed=1,
                       save_history=True,
                       verbose=True)

        return res

    def get_best(self, res):

        for i in range(len(res.F)):

            w_metric = -1 * res.F[:, 0] - res.F[:, 1] - res.F[:, 2]

            if w_metric[i] == max(w_metric[:]):
                best_x = res.X[i]
                best_f = res.F[i]

        return best_x, best_f

In [ ]:
solver = Solvers()
trainer = Trainers()

res = solver.svrSolver(X_train=X_train,
                X_test=X_test,
                y_train=y_train,
                y_test=y_test)

h_params, best_F = solver.get_best(res)

score = trainer.trainSVR(h_params=h_params,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test,
                        folds=5)

print(h_params)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

In [ ]:
#SVR Model with best hyperparamets
encoder_SVR=SVR(C = 14.21384202663237, epsilon= 0.010102095368884755).fit(X_train, y_train)

In [ ]:
#Scores (R^2, MAE, MSE)
#Change the model name accordingly
# Train
(SVR_MF_train_mse_A1, SVR_MF_train_mae_A1,SVR_MF_train_rsquared_A1) = score(encoder_SVR, X_train, y_train)
## Test
(SVR_MF_test_mse_A1, SVR_MF_test_mae_A1,SVR_MF_test_rsquared_A1) = score(encoder_SVR, X_test, y_test)

print(SVR_MF_train_rsquared_A1)
print(SVR_MF_train_mae_A1)
print(SVR_MF_train_mse_A1)

## Activity coefficient predictions versus measured values

In [ ]:
# Change the model name accordingly
plot(encoder_SVR,X_train,y_train,X_test, y_test)

## SHAP Values

In [ ]:
## Change the model accordingly
shap_plot(encoder_SVR,X_train,y_train,X_test, y_test,X)

## Hyper-parameter optimization(ANN regression)

In [ ]:
class annModel(ElementwiseProblem):
    def __init__(self, X_train, X_test, y_train, y_test, **kwargs):
        vars = {
            "alpha": Choice(options = [0.0001, 0.001, 0.01, 0.1, 1]),
            "learning_rate_init": Choice(options = [0.001, 0.01, 0.1, 1.0]),
            "activation_idx":Integer(bounds = (0, 2)),
            'solver_idx': Integer(bounds = (0, 1)),
            "learning_rate_idx": Integer(bounds=(0, 2))
        }


        super().__init__(vars=vars, n_obj=3, n_ieq_constr=0, **kwargs)

        self.trainer = Trainers()
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test


    def _evaluate(self, X, out, *arg, **kwargs):

        scores = self.trainer.trainANN_SingleOutput(h_params= X,
                                       X_train=self.X_train,
                                       X_test=self.X_test,
                                       y_train=self.y_train,
                                       y_test=self.y_test,
                                       folds=5)

        out['F'] = [-1 * scores[0], scores[1], -1 * scores[2]]


class Solvers(ElementwiseProblem):
    def __init__(self):
        super(ElementwiseProblem, self).__init__()

    def annSolver(self, X_train, X_test, y_train, y_test):

        problem = annModel(X_train, X_test, y_train, y_test)

        algorithm = NSGA2(  
                        pop_size=2,
                        sampling=MixedVariableSampling(),
                        mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                        eliminate_duplicates=MixedVariableDuplicateElimination(),
                        n_offsprings=4
                        )


        res = minimize(problem,
                       algorithm,
                       termination=("n_gen",10),
                       seed=1,
                       save_history=True,
                       verbose=True)

        return res

    def get_best(self, res):

        for i in range(len(res.F)):

            w_metric = -1 * res.F[:, 0] - res.F[:, 1] - res.F[:, 2]

            if w_metric[i] == max(w_metric[:]):
                best_x = res.X[i]
                best_f = res.F[i]

        return best_x, best_f

In [ ]:
solver = Solvers()
trainer = Trainers()

res = solver.annSolver(X_train=X_train,
                X_test=X_test,
                y_train=y_train,
                y_test=y_test)

h_params, best_F = solver.get_best(res)

score = trainer.trainANN_SingleOutput(h_params=h_params,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test,
                        folds=5)
print(h_params)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

activation                 = ['logistic', 'tanh', 'relu']
solver                     = ['lbfgs', 'adam']
learning_rate              = ['constant', 'invscaling', 'adaptive']

##ANN Model with best hyperparamets
encoderANN =MLPRegressor(alpha=0.01,learning_rate_init=1.0,activation=activation[1],solver=solver[0],learning_rate=learning_rate[1], max_iter = 5000, n_iter_no_change = 200).fit(X_train, y_train)




## Hyper parameter optimization (random forest regression)

In [ ]:
class rfrModel(ElementwiseProblem):
    def __init__(self, X_train, X_test, y_train, y_test, **kwargs):
        vars = {
            'max_depth': Integer(bounds = (1, 100)),
            'n_estimators':Integer(bounds = (1, 100)),
            'max_features_idx': Integer(bounds = (0, 2)),
            'min_weight_fraction_leaf':Real(bounds = (0, 1)),
            'bootstrap_idx':Integer(bounds = (0,1))
            
            
        }

        super().__init__(vars=vars, n_obj=3, n_ieq_constr=0, **kwargs)

        self.trainer = Trainers()
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test


    def _evaluate(self, X, out, *arg, **kwargs):

        scores = self.trainer.trainRFR(h_params= X,
                                       X_train=self.X_train,
                                       X_test=self.X_test,
                                       y_train=self.y_train,
                                       y_test=self.y_test,
                                       folds=5)

        out['F'] = [-1 * scores[0], scores[1], -1 * scores[2]]


class Solvers(ElementwiseProblem):
    def __init__(self):
        super(ElementwiseProblem, self).__init__()

    def rfrSolver(self, X_train, X_test, y_train, y_test):

        problem = rfrModel(X_train, X_test, y_train, y_test)

        #change the pop-size, n_offsprings, n_gen (if needed)
        algorithm = NSGA2(  
                        pop_size=2,
                        sampling=MixedVariableSampling(),
                        mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                        eliminate_duplicates=MixedVariableDuplicateElimination(),
                        n_offsprings=2
                        )


        res = minimize(problem,
                       algorithm,
                       termination=("n_gen", 200),
                       seed=1,
                       save_history=True,
                       verbose=True)

        return res

    def get_best(self, res):

        for i in range(len(res.F)):

            w_metric = -1 * res.F[:, 0] - res.F[:, 1] - res.F[:, 2]

            if w_metric[i] == max(w_metric[:]):
                best_x = res.X[i]
                best_f = res.F[i]

        return best_x, best_f

In [ ]:
solver = Solvers()
trainer = Trainers()


res = solver.rfrSolver(X_train=X_train,
                X_test=X_test,
                y_train=y_train,
                y_test=y_test)

h_params, best_F = solver.get_best(res)

score = trainer.trainRFR(h_params=h_params,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train,
                        y_test=y_test,
                        folds=5)
max_features = ['auto', 'sqrt','log2']
bootstrap = ['True', 'False']
print(h_params)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

In [ ]:
## RFR Model with best hyperparamets
encoderRFR = RandomForestRegressor(max_depth=27,n_estimators=80,max_features=max_features[0],bootstrap = bootstrap[0],min_weight_fraction_leaf=0.00034777617715460105).fit(X_train, y_train)
encoderRFR